In [188]:
import pandas as pd
from datetime import datetime

excel_path = '/Users/zeynepdundar/Downloads/INDR491_UPDATED.xlsx'
df = pd.read_excel (excel_path)


df_new = df.copy()
timestamp = 1553367060


In [189]:
df_new = df.copy()

# make a plain dataframe containing only date and value columns
df_new_sade = pd.concat([df_new["datetime"], df_new["Value Updated"]], axis=1)



In [190]:
#create groups of dataframe by time intervals and select the rows which are bigger than the threshold

numerator = df_new_sade.groupby(pd.Grouper(key='datetime', offset="2s", freq="180min", label='right')).apply(lambda x: x[x["Value Updated"]>5]["Value Updated"].sum())
denominator = df_new_sade.groupby(pd.Grouper(key='datetime', offset="2s", freq="180min", label='right')).apply(lambda x: x["Value Updated"].sum())

#calculate proportions for every period
proportions = numerator/denominator
proportions 

datetime
2021-11-16 21:00:02    0.0
2021-11-17 00:00:02    0.0
2021-11-17 03:00:02    0.0
2021-11-17 06:00:02    0.0
2021-11-17 09:00:02    0.0
                      ... 
2022-11-15 12:00:02    0.0
2022-11-15 15:00:02    0.0
2022-11-15 18:00:02    0.0
2022-11-15 21:00:02    0.0
2022-11-16 00:00:02    0.0
Freq: 180T, Length: 2914, dtype: float64

In [191]:
# check the nonzero values 
# numerator
# denominator
# proportions
# (numerator[numerator != 0])
# (proportions[proportions != 0])

In [192]:
states = [0]*len(proportions)
states

proportion_df = pd.DataFrame(proportions, columns = ["Proportions"])
proportion_df["State"] = states
proportion_df

,Proportions,State
datetime,,
2021-11-16 21:00:02,0.0,0
2021-11-17 00:00:02,0.0,0
2021-11-17 03:00:02,0.0,0
2021-11-17 06:00:02,0.0,0
2021-11-17 09:00:02,0.0,0
...,...,...
2022-11-15 12:00:02,0.0,0
2022-11-15 15:00:02,0.0,0
2022-11-15 18:00:02,0.0,0


In [194]:
#proportions[proportions["Value Updated"] == 0.951327]["states"]
# filter ederken esit olanlari degistirmiyor
proportion_df.loc[proportion_df["Proportions"] <= 0.3,"State"] = "State 1"
proportion_df.loc[(proportion_df["Proportions"] <= 0.5) & (proportion_df["Proportions"] > 0.3),"State"] = "State 2"
proportion_df.loc[(proportion_df["Proportions"] <= 0.8) & (proportion_df["Proportions"] > 0.5),"State"] = "State 3"
proportion_df.loc[proportion_df["Proportions"] > 0.8,"State"] = "State 4"

# example
proportion_df[proportion_df["State"] == "State 3"]



,Proportions,State
datetime,,
2022-01-14 21:00:02,0.682547,State 3
2022-03-24 00:00:02,0.614477,State 3
2022-04-14 06:00:02,0.762449,State 3
2022-08-19 21:00:02,0.531565,State 3
2022-11-02 21:00:02,0.713589,State 3


In [ ]:
import numpy as np

def getIndexByState(state):
    
    if state == "State 1":
        index = 0
    elif state == "State 2":
        index = 1
    elif state == "State 3":
        index = 2
    elif state == "State 4":
        index = 3
    else:
        index = 0 
    return index


# calculate transitions

prev_index = 0
cur_row_index = 1
number_of_states = 4
transitions = np.zeros((number_of_states,number_of_states))
total_transitions = 0

for index, cur_row in proportion_df.iterrows():
  
    cur_index = getIndexByState(cur_row["State"])
    transitions[prev_index][cur_index] += 1
    prev_index = cur_index
    total_transitions += 1

transition_probs = transitions/total_transitions
transition_probs




array([[9.71173644e-01, 0.00000000e+00, 1.71585450e-03, 2.05902539e-03],
       [3.43170899e-04, 0.00000000e+00, 0.00000000e+00, 3.43170899e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.71585450e-03],
       [3.43170899e-03, 6.86341798e-04, 0.00000000e+00, 1.85312286e-02]])